In [16]:
from random import seed
from random import randrange
from csv import reader
from math import exp

In [17]:
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		data = reader(file)
		for row in data:
			if not row:
				continue
			dataset.append(row)
	return dataset

In [18]:
# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

In [19]:
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_val = [row[i] for row in dataset]
		val_min = min(col_val)
		val_max = max(col_val)
		minmax.append([val_min, val_max])
	return minmax

In [20]:
# Normalize dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [21]:
# Split dataset into k folds
def cross_validation_split(dataset, n):
	split = list()
	copy = list(dataset)
	fold_size = int(len(dataset) / n)
	for i in range(n):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(copy))
			fold.append(copy.pop(index))
		split.append(fold)
	return split

In [22]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [23]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n, *args):
	folds = cross_validation_split(dataset, n)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			copy = list(row)
			test_set.append(copy)
			copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

In [24]:
# Make a prediction with coefficients
def predict(row, coefficients):
	c = coefficients[0]
	for i in range(len(row)-1):
		c += coefficients[i + 1] * row[i]
	return 1.0 / (1.0 + exp(-c))

In [25]:
# Estimate logistic regression coefficients using stochastic gradient descent
def coefficients_sgd(train, l_rate, n):
	coef = [0.0 for i in range(len(train[0]))]
	for epoch in range(n):
		for row in train:
			c = predict(row, coef)
			error = row[-1] - c
			coef[0] = coef[0] + l_rate * error * c * (1.0 - c)
			for i in range(len(row)-1):
				coef[i + 1] = coef[i + 1] + l_rate * error * c * (1.0 - c) * row[i]
	return coef

In [26]:
# Linear Regression Algorithm With Stochastic Gradient Descent
def logistic_regression(train, test, l_rate, n):
	predictions = list()
	coef = coefficients_sgd(train, l_rate, n)
	for row in test:
		c = predict(row, coef)
		c = round(c)
		predictions.append(c)
	return(predictions)

In [27]:
# Test the logistic regression algorithm on the dataset
seed(1)
# load and prepare data
filename = 'pima-indians-diabetes.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])):
	str_column_to_float(dataset, i)
# normalize
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.1
n_epoch = 100
scores = evaluate_algorithm(dataset, logistic_regression, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [73.8562091503268, 78.43137254901961, 81.69934640522875, 75.81699346405229, 75.81699346405229]
Mean Accuracy: 77.124%


In [28]:
import unittest

class TestNotebook(unittest.TestCase):
    def test_evaluate_algo(self):
        scores = evaluate_algorithm(dataset, logistic_regression, n_folds, l_rate, n_epoch)
        accuracy = sum(scores)/float(len(scores))
        threshold = 75
        self.assertGreater(accuracy, threshold)
    
unittest.main(argv=[''], verbosity=2, exit=False)

test_evaluate_algo (__main__.TestNotebook) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.897s

OK
